In [2]:
import sentinelsat
from sentinelsat import SentinelAPI, read_geojson, geojson_to_wkt
from datetime import date
api = SentinelAPI('mbossing', 'Iaintdoin12')

In [9]:
my_geojson = {
  "type": "Feature", #let's define our GeoJSON type. As it's a single geometry, it's just a single 'Feature'
  "geometry": {
	"type": "Polygon", #let's define our geometry type, which as we have a square, is a polygon.
	"coordinates": [ #Here are our actual geometry coordinates
	  [
		[
		  -62.675634,
		  -64.672511
		],
		[
		  -62.675634,
		  -64.668629
		],
		[
		  -62.665516,
		  -64.668629
		],
		[
		  -62.665516,
		  -64.672511
		],
		[
		  -62.675634,
		  -64.672511
		]
	  ],
	]
  },
    "properties": {'id': 'Georges Point'}, #And an example geometry ID, although not strictly necessary for this task
}

footprint = geojson_to_wkt(my_geojson)
footprint

products = api.query(
    footprint,
    platformname = 'Sentinel-2',
    date = ('20100101', '20190101'),
    cloudcoverpercentage = (0,10),
    limit=10
)

# Now download them!!
api.download_all(products)

LTA retrieval:   0%|          | 0/1 [00:00<?, ?product/s]

MD5 checksumming:   0%|          | 0.00/8.84M [00:00<?, ?B/s]

ResultTuple(downloaded={'fb33f934-cd3c-4d0d-bded-1f442e8b9227': {'id': 'fb33f934-cd3c-4d0d-bded-1f442e8b9227', 'title': 'S2A_MSIL1C_20160312T130902_N0201_R052_T20DNP_20160312T130902', 'size': 400181225, 'md5': 'af2bf4f90ca424e3f9bebd0ffef40b52', 'date': datetime.datetime(2016, 3, 12, 13, 9, 2, 30000), 'footprint': 'POLYGON((-63.00040928555121 -64.0266154415217,-63.0004243068931 -65.01189269520637,-60.67214684722246 -64.99375473329084,-60.75447175679198 -64.00925956233145,-63.00040928555121 -64.0266154415217))', 'url': "https://apihub.copernicus.eu/apihub/odata/v1/Products('fb33f934-cd3c-4d0d-bded-1f442e8b9227')/$value", 'Online': True, 'Creation Date': datetime.datetime(2018, 12, 5, 14, 14, 7, 147000), 'Ingestion Date': datetime.datetime(2018, 12, 5, 0, 31, 59, 685000), 'quicklook_url': "https://apihub.copernicus.eu/apihub/odata/v1/Products('fb33f934-cd3c-4d0d-bded-1f442e8b9227')/Products('Quicklook')/$value", 'path': 'S2A_MSIL1C_20160312T130902_N0201_R052_T20DNP_20160312T130902.zip', 

In [11]:
import rasterio
from rasterio.mask import mask

bands = "images/T58CEV_20181229T194519_TCI_10m.jp2"
my_image = rasterio.open(bands)

blue = my_image.read(1)
green = my_image.read(2)
red = my_image.read(3)

In [6]:
import rasterio
from rasterio.mask import mask

bands = "images/T20DNP_20181225T131909_TCI_10m.jp2"

my_image = rasterio.open(bands)

xmin, ymin, xmax, ymax = my_image.bounds
x_range = xmax - xmin
y_range = ymax - ymin
coords = [
    (xmin,   ymin, (xmin + (x_range/2)), (ymin + (y_range/2))), #bounding box 1
    (xmin,   (ymin + (y_range/2)), (xmin + (x_range/2)), ymax), #bounding box 2
    ((xmin + (x_range/2)),   ymin, xmax, (ymin + (y_range/2))), #bounding box 3
    ((xmin + (x_range/2)),   (ymin + (y_range/2)), xmax, ymax), #bounding box 4
]
coord = coords[0]
xmin = coord[0]
ymin = coord [1]
xmax = coord[2]
ymax = coord[3]

In [10]:
for coord in coords:

    xmin = coord[0]
    ymin = coord [1]
    xmax = coord[2]
    ymax = coord[3]

    my_geojson = [{
        "type": "Polygon", #let's define our geometry type, which as we have a square, is a polygon.
        "coordinates": [ #Here are our actual geometry coordinates
          [
            [
              xmin,
              ymin
            ],
            [
              xmax,
              ymin
            ],
            [
              xmax,
              ymax
            ],
            [
              xmin,
              ymax
            ],
            [
              xmin,
              ymin
            ]
          ],
        ]
      }]
    with rasterio.open(bands) as img:
        clipped, transform = mask(img, my_geojson, crop=True)
    meta = my_image.meta.copy()
    meta.update(
        {

            "transform": transform,
            "height":clipped.shape[1],
            "width":clipped.shape[2]
        }
    )

    filename_out = '{}.tif'.format(coord) 

    with rasterio.open(filename_out, 'w', **meta) as dst:
        dst.write(clipped)

    print('Writing complete for {}'.format(filename_out))

Writing complete for (499980.0, 1390240.0, 554880.0, 1445140.0).tif
Writing complete for (499980.0, 1445140.0, 554880.0, 1500040.0).tif
Writing complete for (554880.0, 1390240.0, 609780.0, 1445140.0).tif
Writing complete for (554880.0, 1445140.0, 609780.0, 1500040.0).tif
